### Build a very simple proof reading interface with Neuroglancer

In [1]:
# Setup neuroglancer
import neuroglancer
from IPython.display import display, HTML

# Uncomment the following if you intend to run from a remote server:
# neuroglancer.set_server_bind_address('0.0.0.0')

neuroglancer.set_static_content_source(url="https://neuroglancer-demo.appspot.com/python")
viewer = neuroglancer.Viewer()
display(HTML('Neuroglancer link: <a href="{0}">{0}</a>'.format(viewer.get_viewer_url())))

In [2]:
# Create a 10x10x10 grid of unique labels
import numpy as np
dim = 20
data = np.arange(dim*dim*dim, dtype=np.uint64).reshape(dim, dim, dim)

In [3]:
# Create a new segmentation layer with our labels
with viewer.txn() as s:
        s.layers['labels'] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(data=data, voxel_size=(1, 1, 1)),
        hideSegmentZero = False
    )

In [4]:
# Create an annotation layer to use for proof reading, linked to the label layer
with viewer.txn() as s:
    s.layers['proof'] = neuroglancer.AnnotationLayer(
        linkedSegmentationLayer="labels"
    )

In [5]:
# Try adding points to the proof layer:
# Select the layer with control-click on the name and then choose the "point" tool.
# Add points with control-click.

# Look at which points we have...
print(viewer.state.layers["proof"].annotations)

[{"point": [7.473986625671387, 7.709526538848877, 10], "type": "point", "id": "6c4f6c71bfcdd96e97db606f7d31fd037b054abb", "segments": ["4147"]}, {"point": [8.491974830627441, 7.624693870544434, 10], "type": "point", "id": "6ffa9c76c5fe877be5ed375734c120b581cf0c39", "segments": ["4148"]}, {"point": [8.44955825805664, 9.024428367614746, 10], "type": "point", "id": "030f9905d278b9e6acec10609a5a9eb3768c80e9", "segments": ["4188"]}, {"point": [8.619223594665527, 8.303353309631348, 10], "type": "point", "id": "f7b20f762e9f1f0e64553f8a0564c3f23f23544d", "segments": ["4168"]}]


In [6]:
# Bind a new 'merge' function to identify the chosen labels and create a segment-group
def merge_labels(s):
    segments = set()
    for segment in s.viewer_state.layers["proof"].annotations:
        if segment.segments:
            segments.update(segment.segments)
    with viewer.txn() as txn:
        for segment in segments:
            txn.layers["labels"].equivalences.union(max(segments), segment)
        txn.layers['proof'].annotations = None
    with viewer.config_state.txn() as txn:
        txn.status_messages['merge'] = 'Merged %s segments' % (len(segments))

# viewer.actions.clear()
viewer.actions.add('merge-labels', merge_labels)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keym'] = 'merge-labels'